In [ ]:
# @title AnxietyLightning Launcher (v27.0 - Orchestrator Edition)# @markdown This simplified launcher now calls a dedicated backend script for setup, making it more robust and easier to update.NGROK_TOKEN = "" #@param {type:"string"}
import osimport sysfrom pathlib import Pathimport subprocessimport runpyimport importlib
# --- 0. Pre-flight Checks & Dependencies ---try:    import gradio as grexcept ImportError:    print("⚠️ Gradio not found. Installing...")    subprocess.run([sys.executable, "-m", "pip", "install", "gradio"], check=True)    import gradio as gr
# --- 1. Define Core Paths & Environment ---HOME_DIR = Path('/content')ANXETY_ROOT = HOME_DIR / 'ANXETY'SCRIPTS_DIR = ANXETY_ROOT / 'scripts'LOG_DIR = ANXETY_ROOT / "logs"
os.environ['HOME_DIR'] = str(HOME_DIR)os.environ['ANXETY_ROOT'] = str(ANXETY_ROOT)os.environ['NGROK_TOKEN'] = NGROK_TOKEN
# --- 2. Synchronize Project Files ---print("--- 🚀 Stage 1: Synchronizing Project & Creating Root Directory ---")SETUP_PY_PATH = SCRIPTS_DIR / 'setup.py'SETUP_PY_URL = "https://raw.githubusercontent.com/drf0rk/AnxietyLightning/main/scripts/setup.py"try:    ANXETY_ROOT.mkdir(parents=True, exist_ok=True)    SCRIPTS_DIR.mkdir(parents=True, exist_ok=True)    subprocess.run(["curl", "-sL", SETUP_PY_URL, "-o", str(SETUP_PY_PATH)], check=True, text=True)    subprocess.run([sys.executable, str(SETUP_PY_PATH)], check=True)    print("✅ Project files synchronized.")    LOG_DIR.mkdir(exist_ok=True)    print(f"✅ Log directory ensured at {LOG_DIR}")except Exception as e:    print(f"❌ FATAL: Project synchronization failed. Error: {e}")    sys.exit(1)
# --- 3. Import Synced Modules & Load Data ---print("\n--- 🚀 Stage 2: Loading Synced Modules & Asset Data ---")if str(ANXETY_ROOT) not in sys.path: sys.path.insert(0, str(ANXETY_ROOT))
try:    # Import the new orchestrator and force a reload in case the notebook is re-run    import scripts.orchestrator    importlib.reload(scripts.orchestrator)    from scripts.orchestrator import save_and_launch_generator    print("✅ Orchestrator module loaded.")except ImportError as e:    print(f"❌ FATAL: Failed to import project orchestrator after sync. Error: {e}")    sys.exit(1)
try:    sd15_data = runpy.run_path(str(ANXETY_ROOT / 'scripts/_models-data.py'))    sdxl_data = runpy.run_path(str(ANXETY_ROOT / 'scripts/_xl-models-data.py'))    loras_data_full = runpy.run_path(str(ANXETY_ROOT / 'scripts/_loras-data.py'))    sd15_model_choices = list(sd15_data.get('sd15_model_data', {}).keys())    sd15_vae_choices = list(sd15_data.get('sd15_vae_data', {}).keys())    sd15_controlnet_choices = list(sd15_data.get('controlnet_list', {}).keys())    sd15_lora_choices = list(loras_data_full.get('lora_data', {}).get('sd15_loras', {}).keys())    sdxl_model_choices = list(sdxl_data.get('sdxl_models_data', {}).keys())    sdxl_vae_choices = list(sdxl_data.get('sdxl_vae_data', {}).keys())    sdxl_controlnet_choices = list(sdxl_data.get('controlnet_list', {}).keys())    sdxl_lora_choices = list(loras_data_full.get('lora_data', {}).get('sdxl_loras', {}).keys())    print("✅ Asset data loaded.")except Exception as e:    print(f"❌ FATAL: Could not load asset data files. Error: {e}")    sys.exit(1)
# --- 4. Define Gradio UI ---print("\n--- 🚀 Stage 3: Defining Gradio UI ---")MODERN_LOG_CSS="""<style>#log_output_html{background-color:#0d1117;border:1px solid #30363d;border-radius:8px;padding:12px;font-family:'Monaco','Consolas','Menlo',monospace;color:#c9d1d9;height:400px;overflow-y:auto}#log_output_html .log-line{display:block;animation:fadeIn .5s ease-in-out; white-space: pre-wrap; word-break: break-all;}#log_output_html .log-header{color:#58a6ff;font-weight:700;margin-top:15px;margin-bottom:5px;text-shadow:0 0 5px rgba(88,166,255,.3)}#log_output_html .log-success{color:#3fb950}#log_output_html .log-error{color:#f85149;font-weight:700}#log_output_html .log-warning{color:#d29922;font-weight:700}#log_output_html .log-url{color:#9e87ff;font-weight:700}#log_output_html .log-download{color:#8b949e;font-size:.85em}@keyframes fadeIn{from{opacity:0;transform:translateY(5px)}to{opacity:1;transform:translateY(0)}}</style>"""webui_selection_args = scripts.orchestrator.webui_selection_args
with gr.Blocks(theme=gr.themes.Soft(), css=MODERN_LOG_CSS) as demo:    gr.Markdown("## ⚡ AnxietyLightning Launcher (v27.0 - Orchestrator Edition)")    with gr.Tabs():        with gr.TabItem("1. Setup & Asset Selection"):            with gr.Row():                webui_dropdown = gr.Dropdown(choices=['ReForge', 'Forge', 'A1111', 'ComfyUI', 'Classic', 'SD-UX'], value='ReForge', label="Select WebUI")                sdxl_toggle = gr.Checkbox(label="Use SDXL Models", value=False)            with gr.Accordion("Asset Selection", open=True):                with gr.Row():                    model_checkboxes = gr.CheckboxGroup(choices=sd15_model_choices, label="Checkpoints", interactive=True)                    vae_checkboxes = gr.CheckboxGroup(choices=sd15_vae_choices, label="VAEs", interactive=True)                with gr.Row():                    lora_checkboxes = gr.CheckboxGroup(choices=sd15_lora_choices, label="LoRAs", interactive=True)                    controlnet_checkboxes = gr.CheckboxGroup(choices=sd15_controlnet_choices, label="ControlNets", interactive=True)            with gr.Accordion("Advanced Options", open=False):                args_textbox = gr.Textbox(label="Commandline Arguments", value=webui_selection_args['ReForge'], lines=2, interactive=True)                with gr.Row():                    ngrok_textbox_ui = gr.Textbox(label="NGROK Token", type="password", scale=3, value=NGROK_TOKEN)                    detailed_dl_checkbox = gr.Checkbox(label="Detailed Logs", value=False, scale=1)                with gr.TabItem("2. Launch & Live Log"):            launch_button = gr.Button("Install, Download & Launch", variant="primary")            output_log = gr.HTML(label="Live Log", elem_id="log_output_html")
    # --- Event Handlers ---    def update_asset_choices_local(is_sdxl):        models, vaes, loras, cnets = (sdxl_model_choices, sdxl_vae_choices, sdxl_lora_choices, sdxl_controlnet_choices) if is_sdxl else (sd15_model_choices, sd15_vae_choices, sd15_lora_choices, sd15_controlnet_choices)        return [gr.update(choices=models, value=[]), gr.update(choices=vaes, value=[]), gr.update(choices=loras, value=[]), gr.update(choices=cnets, value=[])]
    def update_args_local(webui_choice_val):        return gr.update(value=webui_selection_args.get(webui_choice_val, ""))
    sdxl_toggle.change(fn=update_asset_choices_local, inputs=sdxl_toggle, outputs=[model_checkboxes, vae_checkboxes, lora_checkboxes, controlnet_checkboxes])    webui_dropdown.change(fn=update_args_local, inputs=webui_dropdown, outputs=args_textbox)        # --- Main Launch Trigger ---    # The button now calls the reloaded function from the external, updatable script    launch_button.click(        fn=save_and_launch_generator,        inputs=[webui_dropdown, sdxl_toggle, model_checkboxes, vae_checkboxes, lora_checkboxes, controlnet_checkboxes, args_textbox, ngrok_textbox_ui, detailed_dl_checkbox],        outputs=[output_log]    )
# --- 5. Launch Gradio ---print("\n--- 🚀 Stage 4: Launching Gradio UI ---")demo.queue().launch(share=True, inline=False, debug=True)